In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# !pip3 install pandas

In [18]:
try:
#     r = requests.get('https://www.kbb.com/honda/?intent=new')
    r = requests.get('https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=ford&maximum_distance=20&mileage_max=&page_size=20&sort=best_match_desc&stock_type=all&year_max=&year_min=&zip=20741')
except HTTPError as e:
    print(e)
    
bs = BeautifulSoup(r.content, 'html.parser')
# tbl = bs.find_all('h2')
# tbl
# pbs = bs.prettify()
# pbs

[<h2 class="sds-heading--3 lead-form-heading">Contact seller</h2>,
 <h2 class="title">2018 Ford Focus SE</h2>,
 <h2 class="title">2018 Ford Focus SE</h2>,
 <h2 class="title">2014 Ford F-150 STX</h2>,
 <h2 class="title">2014 Ford Mustang GT</h2>,
 <h2 class="title">2003 Ford Focus SE</h2>,
 <h2 class="title">2018 Ford Explorer Limited</h2>,
 <h2 class="title">2016 Ford F-250 XLT</h2>,
 <h2 class="title">2020 Ford Expedition XLT</h2>,
 <h2 class="title">2019 Ford Expedition Platinum</h2>,
 <h2 class="title">2018 Ford F-150 XLT</h2>,
 <h2 class="title">2019 Ford Fusion Energi Titanium</h2>,
 <h2 class="title">2018 Ford Expedition Platinum</h2>,
 <h2 class="title">2018 Ford Focus SE</h2>,
 <h2 class="title">2016 Ford F-250 XLT</h2>,
 <h2 class="title">2018 Ford F-150 Platinum</h2>,
 <h2 class="title">2020 Ford F-150 XLT</h2>,
 <h2 class="title">2017 Ford Mustang GT</h2>,
 <h2 class="title">2020 Ford F-150 Raptor</h2>,
 <h2 class="title">2021 Ford F-150 Lariat</h2>,
 <h2 class="title">2017 